# Create Batch Job for Claude API

This notebook creates a batch job for processing multiple prompts through the Claude API.

## Features
- Supports Claude Sonnet 4 and Opus 4 models
- Extended thinking mode (add `-thinking` suffix to model name)
- Automatic system message handling
- Batch metadata saved for result retrieval

## Prerequisites
- Set the `ANTHROPIC_API_KEY` environment variable
- Prepare input JSON file with `keys` and `contexts` arrays

## Input Format
```json
{
  "keys": ["case_001", "case_002", ...],
  "contexts": [
    [{"role": "user", "content": "..."}],
    [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}],
    ...
  ]
}
```

## Output
- Batch metadata saved to `logs/{INPUT_DIR}/{MODEL_NAME}/{INPUT_FILE}_thinking_{True/False}.json`
- Use `retrieve_batch_results_claude.ipynb` to get results after batch completes

In [ ]:
# ============================================================================
# CONFIGURATION - Edit these variables before running
# ============================================================================
from pathlib import Path

# Base directory for data files
BASE_DIR = Path(".")  # Change to your data directory

# Input file settings
INPUT_DIR = "data"  # Directory containing input JSON file
INPUT_FILE = "your_input_file"  # Name without .json extension

# Model settings
# Supported models: claude-sonnet-4-20250514, claude-opus-4-1-20250805, claude-opus-4-20250514
# Add -thinking suffix to enable extended thinking mode (e.g., claude-sonnet-4-20250514-thinking)
MODEL_NAME = "claude-sonnet-4-20250514-thinking"

# Output directory for logs
LOGS_DIR = BASE_DIR / "logs"

# ============================================================================

In [ ]:
import json
import os
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

# Detect thinking mode and get actual model name
thinking_enabled = MODEL_NAME.endswith("-thinking")
actual_model = MODEL_NAME[:-len("-thinking")] if thinking_enabled else MODEL_NAME
thinking_budget = 16000 if thinking_enabled else None

# Supported Claude 4 models (don't use top_p with these)
CLAUDE_4_MODELS = ["claude-sonnet-4-20250514", "claude-opus-4-1-20250805", "claude-opus-4-20250514"]

# Initialize Anthropic client
client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# Build input path
input_path = BASE_DIR / INPUT_DIR / f"{INPUT_FILE}.json"

# Load the input prompt data
with open(input_path, 'r') as f:
    data = json.load(f)

print(f"Input file: {input_path}")
print(f"Loaded {len(data['keys'])} cases")
print(f"Model: {actual_model}")
print(f"Thinking enabled: {thinking_enabled}")
if thinking_enabled:
    print(f"Thinking budget: {thinking_budget} tokens")

In [ ]:
# Build list of Request objects for batch API
# NOTE: Unlike OpenAI, Anthropic accepts requests directly - no JSONL file upload needed!

requests = []
key_dict = {}

for i, (key, context) in enumerate(list(zip(data["keys"], data["contexts"]))):
    # Handle system message
    messages = context.copy()
    system_content = None
    
    if messages and messages[0]["role"] == "system":
        # Extract system message and remove from messages array
        system_content = messages[0]["content"]
        messages = messages[1:]
    
    # Build params dict for MessageCreateParamsNonStreaming
    params_dict = {
        "model": actual_model,
        "max_tokens": 20000,
        "temperature": 1.0 if thinking_enabled else 0.7,
        "messages": messages,
    }
    
    # Add system parameter if system message exists
    if system_content:
        params_dict["system"] = system_content
    
    # Add thinking parameter if enabled
    if thinking_enabled:
        params_dict["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget
        }
    
    # Create Request object
    batch_request = Request(
        custom_id=str(i),
        params=MessageCreateParamsNonStreaming(**params_dict)
    )
    
    key_dict[str(i)] = key
    requests.append(batch_request)

print(f"Created {len(requests)} batch requests")

In [ ]:
# Create the batch - pass requests directly (no file upload!)
message_batch = client.messages.batches.create(
    requests=requests
)

print(f"Batch created successfully!")
print(f"Batch ID: {message_batch.id}")
print(f"Status: {message_batch.processing_status}")
print(f"\nFull batch info:")
print(message_batch)

In [ ]:
# Check batch status (re-run this cell to check progress)
message_batch = client.messages.batches.retrieve(message_batch.id)

print(f"Batch ID: {message_batch.id}")
print(f"Status: {message_batch.processing_status}")
print(f"\nRequest counts:")
print(f"  Processing: {message_batch.request_counts.processing}")
print(f"  Succeeded: {message_batch.request_counts.succeeded}")
print(f"  Errored: {message_batch.request_counts.errored}")
print(f"  Canceled: {message_batch.request_counts.canceled}")
print(f"  Expired: {message_batch.request_counts.expired}")

if message_batch.processing_status == "ended":
    print(f"\nBatch completed!")
    if message_batch.results_url:
        print(f"Results URL: {message_batch.results_url}")
elif message_batch.processing_status == "in_progress":
    print(f"\nBatch still processing...")
else:
    print(f"\nStatus: {message_batch.processing_status}")

In [ ]:
# Save batch metadata and key_dict for later result retrieval

# Convert batch object to dict
try:
    message_batch_dict = message_batch.to_dict()
except AttributeError:
    message_batch_dict = dict(message_batch)

# Build log save path
thinking_suffix = "_thinking_True" if thinking_enabled else ""
log_save_path = LOGS_DIR / INPUT_DIR / MODEL_NAME / f"{INPUT_FILE}{thinking_suffix}.json"

# Create directory if it doesn't exist
log_save_path.parent.mkdir(parents=True, exist_ok=True)

# Save batch metadata and key mapping
saving_dict = {
    "message_batch": message_batch_dict,
    "key_dict": key_dict
}

with open(log_save_path, 'w') as f:
    json.dump(saving_dict, f, indent=4, default=str)

print(f"Batch metadata saved to: {log_save_path}")
print(f"\nBatch ID: {message_batch.id}")
print(f"\nTo check status later, use:")
print(f"  client.messages.batches.retrieve('{message_batch.id}')")
print(f"\nTo retrieve results when complete, use retrieve_batch_results_claude.ipynb")

In [ ]:
# Optional: List recent batches
print("Recent batches:\n")
for i, batch in enumerate(client.messages.batches.list(limit=10)):
    print(f"{i+1}. ID: {batch.id}")
    print(f"   Status: {batch.processing_status}")
    print(f"   Created: {batch.created_at}")
    print()